In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import ast
%matplotlib inline
sns.set_theme()

In [ ]:
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [ ]:
from src.strategy import predict_position
from src.update_files import update_state, update_portfolio
from src.position_sizing import portfolio_calculator, amount_calculator
from src.risk_management import slippage, stop_loss, take_profit
from src.backtester import data_handler, place_order, execute_order, execute_SL_TP
from src.utils import get_config, read_file

In [ ]:
config = get_config.read_yaml()
symbols = config['data']['symbols']
symbols

In [ ]:
test_data = read_file.read_merged_test_data()
test_data

In [ ]:
update_state.set_state(test_data, 100000)
state = read_file.read_state()
state

In [ ]:
candle = data_handler.fetch_data(test_data)
candle

In [ ]:
execute_SL_TP.execute(candle)

In [ ]:
candle = predict_position.predict(candle)
candle

In [ ]:
update_portfolio.set_portfolio()
portfolio = read_file.read_portfolio()
portfolio

In [ ]:
Pt = portfolio_calculator.calculate(candle)
Pt

In [ ]:
candle = slippage.get_order_price(candle, Pt)
candle

In [ ]:
candle = amount_calculator.calculate(candle, Pt)
candle

In [ ]:
candle = stop_loss.get_stop_loss(candle)
candle

In [ ]:
candle = take_profit.get_take_profit(candle)
candle

In [ ]:
order = place_order.place(candle)
order

In [ ]:
execute_order.execute(order)
portfolio = read_file.read_portfolio()
portfolio

In [ ]:
state = read_file.read_state()
state

In [ ]:
candle2 = data_handler.fetch_data(test_data)
candle2

In [ ]:
# execute sltp
execute_SL_TP.execute(candle2)

In [ ]:
candle2 = predict_position.predict(candle2)
candle2

In [ ]:
Pt = portfolio_calculator.calculate(candle)
Pt

In [ ]:
candle2 = slippage.get_order_price(candle2, Pt)
candle2

In [ ]:
candle2 = amount_calculator.calculate(candle2, Pt)
candle2

In [ ]:
candle2 = stop_loss.get_stop_loss(candle2)
candle2 = take_profit.get_take_profit(candle2)
candle2

In [ ]:
order2 = place_order.place(candle2)
order2

In [ ]:
execute_order.execute(order2)
portfolio = read_file.read_portfolio()
portfolio

In [ ]:
state = read_file.read_state()
state